<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#处理方法" data-toc-modified-id="处理方法-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>处理方法</a></span><ul class="toc-item"><li><span><a href="#颜色处理" data-toc-modified-id="颜色处理-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>颜色处理</a></span></li><li><span><a href="#降噪" data-toc-modified-id="降噪-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>降噪</a></span></li><li><span><a href="#去除边框" data-toc-modified-id="去除边框-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>去除边框</a></span></li><li><span><a href="#字符切割" data-toc-modified-id="字符切割-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>字符切割</a></span></li></ul></li><li><span><a href="#图形ORC" data-toc-modified-id="图形ORC-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>图形ORC</a></span></li><li><span><a href="#识别库" data-toc-modified-id="识别库-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>识别库</a></span><ul class="toc-item"><li><span><a href="#tesserocr" data-toc-modified-id="tesserocr-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>tesserocr</a></span></li></ul></li></ul></div>

# 处理方法
1. 灰度处理、二值化
2. 去除边框（如果有的话）
3. 降噪
4. 切割字符或者倾斜度矫正
5. 训练字体库
6. 识别

https://www.cnblogs.com/qqandfqr/p/7866650.html

## 颜色处理
- 灰度处理：把彩色的图片转为灰色的图片

- 二值化：是将图片处理为只有黑白两色


In [22]:
# pillow
from PIL import Image
import numpy as np

def get_binary_image(img_name):
    img = Image.open(img_name)
    img_mat = np.array(img)                           # 图片 → 矩阵
    
    img = img.convert('L')                 # 转变成灰度
    
    img_mat = (img_mat > 100) * 255        # 二值化（设置阈值100，大于的全变为白色）
    
    img = Image.fromarray(img_mat.astype('uint8'))    # 矩阵 → 图片
    
    return img

In [21]:
# cv
def _get_dynamic_binary_image(filedir, img_name):
    filename =   './out_img/' + img_name.split('.')[0] + '-binary.jpg'
    img_name = filedir + '/' + img_name
    print('.....' + img_name)
    im = cv2.imread(img_name)
    #灰值化
    im = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY) 
    # 二值化
    th1 = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 1)
    cv2.imwrite(filename,th1)
    return th1

## 降噪
- 点降噪
- 线降噪

过程：检测这个点相邻的四个点，判断这四个点中是白点的个数，如果有两个以上的白色像素点，那么就认为这个点是白色的

限制：如果干扰线特别粗就没有办法去除，只能去除细的干扰线

In [10]:
# 干扰线降噪
def interference_line(img, img_name):
    filename =  './out_img/' + img_name.split('.')[0] + '-interferenceline.jpg'
    h, w = img.shape[:2]    # opencv矩阵点是反的
    for y in range(1, w - 1):
        for x in range(1, h - 1):
            count = 0
            if img[x, y - 1] > 245:
                count = count + 1
            if img[x, y + 1] > 245:
                count = count + 1
            if img[x - 1, y] > 245:
                count = count + 1
            if img[x + 1, y] > 245:
                count = count + 1
            if count > 2:
                img[x, y] = 255
    cv2.imwrite(filename,img)
    return img

## 去除边框
去遍历像素点，找到四个边框上的所有点(宽度)，把他们都改为白色

In [9]:
def clear_border(img,img_name):
    filename = './out_img/' + img_name.split('.')[0] + '-clearBorder.jpg'
    h, w = img.shape[:2]
    
    for y in range(0, w):
        for x in range(0, h):
            if y < 2 or y > w - 2:    # 把外出层两2pix的边框去掉
                img[x, y] = 255
            if x < 2 or x > h - 2:
                img[x, y] = 255
        cv2.imwrite(filename,img)
    return img

## 字符切割
将粘连的字符切割为单个的字符，再进行识别

过程：
1. 找到一个黑色的点，遍历与他相邻的黑色的点，找出这些点中的最高、最低、最右、最左的点，记录下这四个点，认为这是一个字符。
2. 再向后遍历点，直至找到黑色的点，继续步骤1

<img src='img/clip.png' width=400>



In [17]:
def cfs(im,x_fd,y_fd):
    '''
    用队列和集合记录遍历过的像素坐标代替单纯递归以解决cfs访问过深问题
    '''

    xaxis=[]
    yaxis=[]
    visited =set()
    q = Queue()
    q.put((x_fd, y_fd))
    visited.add((x_fd, y_fd))
    offsets=[(1, 0), (0, 1), (-1, 0), (0, -1)]    #四邻域

    while not q.empty():
        x,y=q.get()

        for xoffset,yoffset in offsets:
            x_neighbor,y_neighbor = x+xoffset,y+yoffset
            
            if (x_neighbor,y_neighbor) in (visited):
                continue  # 已经访问过了

            visited.add((x_neighbor, y_neighbor))

            try:
                if im[x_neighbor, y_neighbor] == 0:
                    xaxis.append(x_neighbor)
                    yaxis.append(y_neighbor)
                    q.put((x_neighbor,y_neighbor))
            except IndexError:
                pass
    # print(xaxis)
    if (len(xaxis) == 0 | len(yaxis) == 0):
        xmax = x_fd + 1
        xmin = x_fd
        ymax = y_fd + 1
        ymin = y_fd

    else:
        xmax = max(xaxis)
        xmin = min(xaxis)
        ymax = max(yaxis)
        ymin = min(yaxis)
    #ymin,ymax=sort(yaxis)

    return ymax,ymin,xmax,xmin

    def detectFgPix(im,xmax):
        '''
        搜索区块起点
        '''

    h,w = im.shape[:2]
    for y_fd in range(xmax+1,w):
        for x_fd in range(h):
            if im[x_fd,y_fd] == 0:
                return x_fd,y_fd

    def CFS(im):
        '''
        切割字符位置
        '''

    zoneL=[]#各区块长度L列表
    zoneWB=[]#各区块的X轴[起始，终点]列表
    zoneHB=[]#各区块的Y轴[起始，终点]列表

    xmax=0#上一区块结束黑点横坐标,这里是初始化
    for i in range(10):

        try:
            x_fd,y_fd = detectFgPix(im,xmax)
            # print(y_fd,x_fd)
            xmax,xmin,ymax,ymin=cfs(im,x_fd,y_fd)
            L = xmax - xmin
            H = ymax - ymin
            zoneL.append(L)
            zoneWB.append([xmin,xmax])
            zoneHB.append([ymin,ymax])

        except TypeError:
            return zoneL,zoneWB,zoneHB

    return zoneL,zoneWB,zoneHB

In [18]:
def cutting_img(im,im_position,img,xoffset = 1,yoffset = 1):
    filename =  './out_img/' + img.split('.')[0]
    # 识别出的字符个数
    im_number = len(im_position[1])
    # 切割字符
    for i in range(im_number):
        im_start_X = im_position[1][i][0] - xoffset
        im_end_X = im_position[1][i][1] + xoffset
        im_start_Y = im_position[2][i][0] - yoffset
        im_end_Y = im_position[2][i][1] + yoffset
        cropped = im[im_start_Y:im_end_Y, im_start_X:im_end_X]
        cv2.imwrite(filename + '-cutting-' + str(i) + '.jpg',cropped)

使用库：Pillow(python图像处理库)、OpenCV(高级图像处理库)、pytesseract(识别库)


# 图形ORC
OCR( Optical Character Recognition，光学字符识别) ：扫描字符，通过其形状将其翻译成电子文本的过程。

对于图形验证码来说，它们都是一些不规则的字符，是由字符稍加扭曲变换得到的内容。


# 识别库


## tesserocr 
tesserocr 是一个 OCR 识别库，是对 tesseract 做的一层 Python API 封装，所以它的核心是 tesseract



In [2]:
# 导入包
import locale
locale.setlocale(locale.LC_ALL, 'C')    # 设置当前运行环境
import tesserocr

from PIL import Image

image = Image.open('img/test.jpg')       # 传入图片
print(tesserocr.image_to_text(image))    # OCR识别

Python3WebSpider

